### 10.1) Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### 10.2) Define Gaussian distribution function

In [ ]:
def gaussian(x, mean, std):
    return np.exp(-np.power(x - mean, 2.) / (2 * np.power(std, 2.)))

### 10.3) Define and visualize target function

In [ ]:
np.random.seed(3)

time = np.linspace(0, 100, 500)
P = np.random.randint(1, 100, 7)

f_x = P[0] + gaussian(time, P[1], P[2]) + gaussian(time, P[3], P[4]) + gaussian(time, P[5], P[6])

f_x_plus_noise = f_x + np.random.normal(0, 0.1, 500)

plt.figure()
plt.plot(time, f_x, lw=2, label='Target Function')
plt.plot(time, f_x_plus_noise, zorder=0, label='Target plus Noise')
plt.legend()
plt.show()

print('Target Function Parameter Values: ', P)

### 10.4) Define Fitness Function

In [ ]:
def calculate_fitness(target, query):
    
    euclidean_distance = sum((target-query)**2) / len(target)
    
    fitness = 1 / (1 + euclidean_distance)
    
    return fitness

### 10.5) Check fitness of target funciton and target function plus noise

In [ ]:
fitness = calculate_fitness(f_x, f_x_plus_noise)

print(fitness)

### 10.6) Initialize ten random models

In [ ]:
np.random.seed(0)
models = np.random.randint(1, 100, (10, 7))

print(models)

### 10.7) Define Cross-Over function (and check it)

In [ ]:
def cross_over(model_1, model_2):
    
    cross_point = np.random.randint(1, len(model_1) - 1)
    
    model_12 = np.concatenate([model_1[:cross_point], model_2[cross_point:]])
    
    model_21 = np.concatenate([model_2[:cross_point], model_1[cross_point:]])
    
    return model_12, model_21


crossed_1, crossed_2 = cross_over(models[0], models[1])
print(crossed_1)
print(crossed_2)

### 10.8) Define Point Mutation function (and check it)

In [ ]:
def point_mutation(model):
    
    mutation_point = np.random.randint(0, len(model))
    
    mutated_model = model.copy()
    
    mutated_model[mutation_point] = np.random.randint(1, 100)
    
    return mutated_model
    

mutated_1 = point_mutation(models[0])
print(mutated_1)

### 10.9) Define Model Ranking function (and check it)

In [ ]:
def rank_models(models, fitness_function):
    
    fitness_scores = []
    
    for i in range(models.shape[0]):
        
        P = models[i]

        f_x_model_i = P[0] + gaussian(time, P[1], P[2]) + gaussian(time, P[3], P[4]) + gaussian(time, P[5], P[6])
        
        fitness_score = calculate_fitness(f_x_plus_noise, f_x_model_i)
        
        fitness_scores.append((models[i], fitness_score))
    
    fitness_scores = sorted(fitness_scores, key=lambda tup: tup[1], reverse=True)
    
    ranked_models, ranked_scores = zip(*fitness_scores)
    
    ranked_models, ranked_scores = np.array(ranked_models), np.array(ranked_scores)
    
    return ranked_models, ranked_scores


gen_1_ranked, gen_1_scores = rank_models(models, calculate_fitness)

print(gen_1_ranked)
print('Fitnesses: ', gen_1_scores)

### 10.10) Define Replication function (and check it)

In [ ]:
def replicate(ranked_gen):
    
    next_gen = np.zeros(ranked_gen.shape)
    
    remove_indices = np.arange(int(ranked_gen.shape[0]/2), ranked_gen.shape[0])    
    models_to_reproduce = np.delete(ranked_gen, remove_indices, axis=0)
    
    # copy the top-ranked 1/5 of models
    copy_indices = np.arange(0, int(next_gen.shape[0] * 0.2))
    next_gen[copy_indices] = models_to_reproduce[copy_indices]
    
    # apply cross-over 
    idx = copy_indices[-1] + 1
    i = 0
    
    while idx < int(next_gen.shape[0] * 0.75):
        
        model_1, model_2 = cross_over(models_to_reproduce[i], models_to_reproduce[i + 1])
            
        next_gen[idx] = model_1
        next_gen[idx + 1] = model_2
        
        i += 1
        idx += 2   
    
    
    # apply point mutations
    while idx < next_gen.shape[0]:
        
        random_index = np.random.randint(0, models_to_reproduce.shape[0])
        
        next_gen[idx] = point_mutation(models_to_reproduce[random_index])
        
        idx += 1
        
    
    return next_gen
    

gen_2 = replicate(gen_1_ranked)
print(gen_2)

### 10.11) Run Genetic Algorithm in a loop

In [ ]:
max_generations = 500
fitness_threshold = 0.99

gen = 0
max_fitness = 0

np.random.seed(0)
models = np.random.randint(1, 100, (100, 7)) # Randomly initialize 100 models

while (gen < max_generations) and (max_fitness < fitness_threshold):
    
    gen_ranked, gen_scores = rank_models(models, calculate_fitness)
    
    max_fitness = gen_scores[0]
    
    print('Generation %i Maximum Fitness: %.3f' % (gen, max_fitness))
    
    models = replicate(gen_ranked)
    
    gen += 1


print('Best model parameters: ', models[0])

### 10.12) Plot best model solution beside originating functions

In [ ]:
time = np.linspace(0, 100, 500)

bm = models[0] # best model

print('Best Score: ', gen_scores[0])

bm_x = bm[0] + gaussian(time, bm[1], bm[2]) + gaussian(time, bm[3], bm[4]) + gaussian(time, bm[5], bm[6])

plt.figure()
plt.plot(time, f_x, lw=2, label='Target Function')
plt.plot(time, f_x_plus_noise, zorder=0, label='Target plus Noise')
plt.plot(time, bm_x, lw=2, label='Best Model')
plt.legend()
plt.show()

print('Target Parameter Values: ', P)
print('Solution Parameter Values: ', bm.astype('int'))